## Import Dependencies and set working directory

In [121]:
import json
import os
import pandas as pd
import numpy as np
from datetime import datetime

In [122]:
%cd "C:\Users\lbrig\Documents\GRAD SCHOOL\Semester 3\Covid19 Kaggle\CORD-19-research-challenge"

C:\Users\lbrig\Documents\GRAD SCHOOL\Semester 3\Covid19 Kaggle\CORD-19-research-challenge


## For loop to extract relevant data from each json file

In [123]:
kaggle= []

start=datetime.now()

jsonpaths = ["biorxiv_medrxiv\\biorxiv_medrxiv", "comm_use_subset\\comm_use_subset", "custom_license\\custom_license", "noncomm_use_subset\\noncomm_use_subset"]
for path in jsonpaths:
    for file in os.listdir(path):
        filename = "%s\%s" % (path, file)
        with open(filename, 'r') as covidfile:
            df=json.load(covidfile)

        #Pull out columns needed
        #Paper ID number
        paperid = df['paper_id']
        #Title of paper
        title = df['metadata']['title']
        #Abstract of paper with each portion of text concatinated together
        abstract= ''
        for words in df['abstract']:
            abstract = abstract + ' ' + words['text']
        #Full document with each portion of text concatinated together
        fulldoc= ''
        for words in df['body_text']:
            fulldoc = fulldoc + ' ' + words['text']
        #File type 
        filetype= path.split('\\')
        
        #Append to kaggle 
        kaggle.append({'ID': paperid, 'Title': title, 'Abstract': abstract, 'Document': fulldoc, 'Filepath': filetype[0]})


kaggledf=pd.DataFrame(kaggle)

end=datetime.now()
print("Run time: ", end-start)


Run time:  0:01:21.645252


In [149]:
kaggledf.head(10)

,ID,Title,Abstract,Document,Filepath
0,0015023cc06b5362d332b3baf348d11567ca2fbb,The RNA pseudoknots in foot-and-mouth disease ...,word count: 194 22 Text word count: 5168 23 2...,"VP3, and VP0 (which is further processed to V...",biorxiv_medrxiv
1,004f0f8bb66cf446678dc13cf2701feec4f36d76,Healthcare-resource-adjusted vulnerabilities t...,NaN,The 2019-nCoV epidemic has spread across Chin...,biorxiv_medrxiv
2,00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,"Real-time, MinION-based, amplicon sequencing f...",Infectious bronchitis (IB) causes significant...,"Infectious bronchitis (IB), which is caused b...",biorxiv_medrxiv
3,0139ea4ca580af99b602c6435368e7fdbefacb03,A Combined Evidence Approach to Prioritize Nip...,Nipah Virus (NiV) came into limelight recentl...,Nipah is an infectious negative-sense single-...,biorxiv_medrxiv
4,013d9d1cba8a54d5d3718c229b812d7cf91b6c89,Assessing spread risk of Wuhan novel coronavir...,Background: A novel coronavirus (2019-nCoV) e...,"In December 2019, a cluster of patients with ...",biorxiv_medrxiv
5,01d162d7fae6aaba8e6e60e563ef4c2fca7b0e18,"TWIRLS, an automated topic-wise inference meth...",Faced with the current large-scale public hea...,The sudden outbreak of the new coronavirus (S...,biorxiv_medrxiv
6,01e3b313e78a352593be2ff64927192af66619b5,Title: Viruses are a dominant driver of protei...,Viruses interact with hundreds to thousands o...,The copyright holder for this preprint (which...,biorxiv_medrxiv
7,02201e4601ab0eb70b6c26480cf2bfeae2625193,The impact of regular school closure on season...,School closure is often considered as an opti...,they affect seasonal influenza dynamics. The ...,biorxiv_medrxiv
8,0255ea4b2f26a51a3bfa3bd8f3e1978c82c976d5,Carbon Nanocarriers Deliver siRNA to Intact Pl...,Post-transcriptional gene silencing (PTGS) is...,siRNA without relying on plant transformation...,biorxiv_medrxiv
9,029c1c588047f1d612a219ee15494d2d19ff7439,Protective Population Behavior Change in Outbr...,NaN,"estimation is complicated, however, by effort...",biorxiv_medrxiv


In [125]:
kaggledf.shape

(33224, 5)

In [145]:
#Replacing the cells with only white space with na
kaggledf = kaggledf.replace(r'^\s*$', np.nan, regex=True)

In [146]:
#Count of missing values per column
kaggledf.isna().sum()

ID             0
Title       3649
Abstract    8528
Document       0
Filepath       0
dtype: int64

In [148]:
#How many of each document type?
pd.value_counts(kaggledf.Filepath)


custom_license        20657
comm_use_subset        9315
noncomm_use_subset     2350
biorxiv_medrxiv         902
Name: Filepath, dtype: int64

# Looking at Metadata csv and merging with kaggledf

In [127]:
#New directory
%cd "C:\Users\lbrig\Documents\GRAD SCHOOL\Semester 3\Covid19 Kaggle"

C:\Users\lbrig\Documents\GRAD SCHOOL\Semester 3\Covid19 Kaggle


In [128]:
meta=pd.read_csv("metadata.csv", usecols= [1, 3, 8, 10, 14, 15])

In [129]:
meta.shape

(45774, 6)

In [130]:
#Split the IDs into multiple columns
m2= meta.sha.str.split(";", expand=True)

In [131]:
#Add a prefix to these columns 
m2=m2.add_prefix('ID_')

In [132]:
#Concat the ID columns and metadata
result = pd.concat([m2, meta], axis=1, sort=False)

In [133]:
result.shape

(45774, 12)

In [134]:
#Drop columns
result=result.drop(columns=['sha', 'ID_1', 'ID_2', 'ID_3', 'ID_4', 'ID_5'])

In [135]:
merged= result.merge(kaggledf, how= 'left', left_on='ID_0', right_on= 'ID')

In [136]:
merged.shape

(45774, 11)

In [137]:
merged.tail()

,ID_0,title,abstract,authors,has_full_text,full_text_file,ID,Title,Abstract,Document,Filepath
45769,289deae0b2050aa259a05ba84565a4df82fa099a,Personal Protective Equipment: Protecting Heal...,Abstract Purpose The recent Ebola epidemic tha...,"Fischer, William A.; Weber, David J.; Wohl, Da...",True,custom_license,289deae0b2050aa259a05ba84565a4df82fa099a,Personal Protective Equipment: Protecting Heal...,Purpose: The recent Ebola epidemic that devas...,The recent Ebola epidemic that devastated Wes...,custom_license
45770,21a4369f83891bf6975dd916c0aa495d5df8709e,Viruses and asthma,Abstract Background Viral respiratory infectio...,"Dulek, Daniel E.; Peebles, R. Stokes",True,custom_license,21a4369f83891bf6975dd916c0aa495d5df8709e,Viruses and asthma ☆,Background: Viral respiratory infection has l...,Asthma is a complex disease regulated by both...,custom_license
45771,NaN,Why the WHO won't use the p-word,"There are no criteria for a pandemic, but covi...","MacKenzie, Debora",False,custom_license,NaN,NaN,NaN,NaN,NaN
45772,NaN,"Communication, transparency key as Canada face...",NaN,"Glauser, Wendy",False,NaN,NaN,NaN,NaN,NaN,NaN
45773,3369a14e1d116943f48b3a33597796c9802de279,Searching for animal models and potential targ...,Emerging and re-emerging pathogens represent a...,"Vergara-Alert, Júlia; Vidal, Enric; Bensaid, A...",True,noncomm_use_subset,3369a14e1d116943f48b3a33597796c9802de279,Searching for animal models and potential targ...,Emerging and re-emerging pathogens represent ...,Searching for animal models and potential tar...,noncomm_use_subset


In [138]:
pd.value_counts(merged.has_full_text)

True     31753
False    14021
Name: has_full_text, dtype: int64